In [1]:
# Imports and general data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import re

from sklearn import preprocessing
from scipy.stats import chi2_contingency
from matplotlib.backends.backend_pdf import PdfPages


# hide ipykernel warnings 
import warnings
warnings.filterwarnings('ignore')

#Plotting inline
%matplotlib inline

In [2]:
# Import necessary data
leavetimes_df = pd.read_csv('trim_leavetimes.csv', sep=';') # Choose only relevant columns
trips_df = pd.read_csv('trim_trips.csv', sep=';') #choose only relevant columns
distance_df = pd.read_csv('distance_between_stops.csv')
weather_df = pd.read_csv('Weather_2018.csv') #choose only relevant columns
stops_df = pd.read_csv('num_stops.csv')

In [3]:
leavetimes_df.head()

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,15-JAN-18 00:00:00,6114847,40,60,39356,39356,39469,39482,1000912,NaN,NaN,NaN,NaN,NaN,NaN,23-JAN-18 10:24:03,NaN
1,DB,15-JAN-18 00:00:00,6119911,40,60,44156,44156,44246,44259,1000183,NaN,NaN,NaN,NaN,NaN,NaN,23-JAN-18 10:24:03,NaN
2,DB,15-JAN-18 00:00:00,6107234,40,60,45356,45356,45172,45188,2868369,NaN,NaN,NaN,NaN,NaN,NaN,23-JAN-18 10:24:03,NaN
3,DB,15-JAN-18 00:00:00,6107787,40,60,41756,41756,41719,41734,2693269,NaN,NaN,NaN,NaN,NaN,NaN,23-JAN-18 10:24:03,NaN
4,DB,15-JAN-18 00:00:00,6109346,40,60,40556,40556,40355,40373,1001137,NaN,NaN,NaN,NaN,NaN,NaN,23-JAN-18 10:24:03,NaN


In [4]:
trips_df.dtypes

DATASOURCE          object
DAYOFSERVICE        object
TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
BASIN               object
TENDERLOT          float64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE                object
dtype: object

In [5]:
# Adjust times from seconds to hh:mm:ss and get the expected and actual trip time
#trips_df['PLANNEDTIME'] = (trips_df['PLANNEDTIME_ARR'] - trips_df['PLANNEDTIME_DEP'])
trips_df['ACTUALTIME_SECONDS'] = (trips_df['ACTUALTIME_ARR'] - trips_df['ACTUALTIME_DEP'])
leavetimes_df['DWELL_TIME'] = (leavetimes_df['ACTUALTIME_DEP'] - leavetimes_df['ACTUALTIME_ARR'])
#trips_df['ACTUALTIME_SECONDS'] = trips_df['ACTUALTIME']

trips_df['DATE'] = pd.to_datetime(trips_df['DAYOFSERVICE']).dt.strftime('%Y-%m-%d')
trips_df['HOUR'] = pd.to_datetime(trips_df['PLANNEDTIME_DEP'], unit='s').dt.strftime("%H:00:00")
trips_df['DUMMY_DATE'] = pd.to_datetime(trips_df['DAYOFSERVICE']).dt.strftime('%Y%m%d')
leavetimes_df['DUMMY_DATE'] = pd.to_datetime(leavetimes_df['DAYOFSERVICE']).dt.strftime('%Y%m%d')

#trips_df['UNIQUEID'] = trips_df["DUMMY_DATE"] + "_" + str(trips_df["TRIPID"])
trips_df['UNIQUEID'] = trips_df["DUMMY_DATE"].astype(str) + "_" + trips_df["TRIPID"].astype(str)
leavetimes_df['UNIQUEID'] = leavetimes_df["DUMMY_DATE"].astype(str) + "_" + leavetimes_df["TRIPID"].astype(str)

#trips_df['PLANNEDTIME_ARR'] = pd.to_datetime(trips_df['PLANNEDTIME_ARR'], unit='s').dt.strftime("%H:%M:%S")
#trips_df['PLANNEDTIME_DEP'] = pd.to_datetime(trips_df['PLANNEDTIME_DEP'], unit='s').dt.strftime("%H:%M:%S")
#trips_df['ACTUALTIME_ARR'] = pd.to_datetime(trips_df['ACTUALTIME_ARR'], unit='s').dt.strftime("%H:%M:%S")
#trips_df['ACTUALTIME_DEP'] = pd.to_datetime(trips_df['ACTUALTIME_DEP'], unit='s').dt.strftime("%H:%M:%S")


#leavetimes_df['PLANNEDTIME_ARR'] = pd.to_datetime(leavetimes_df['PLANNEDTIME_ARR'], unit='s').dt.strftime("%H:%M:%S")
#leavetimes_df['PLANNEDTIME_DEP'] = pd.to_datetime(leavetimes_df['PLANNEDTIME_DEP'], unit='s').dt.strftime("%H:%M:%S")
#leavetimes_df['ACTUALTIME_ARR'] = pd.to_datetime(leavetimes_df['ACTUALTIME_ARR'], unit='s').dt.strftime("%H:%M:%S")
#leavetimes_df['ACTUALTIME_DEP'] = pd.to_datetime(leavetimes_df['ACTUALTIME_DEP'], unit='s').dt.strftime("%H:%M:%S")

#trips_df['PLANNEDTIME'] = pd.to_datetime(trips_df['PLANNEDTIME'], unit='s').dt.strftime("%H:%M:%S")
#trips_df['ACTUALTIME'] = pd.to_datetime(trips_df['ACTUALTIME'], unit='s').dt.strftime("%H:%M:%S")


weather_df['DATE'] = pd.to_datetime(weather_df['dt'], unit='s').dt.strftime("%Y-%m-%d")
weather_df['HOUR'] = pd.to_datetime(weather_df['dt'], unit='s').dt.strftime("%H:%M:%S")


In [6]:
# Combine data tables and keep only the relevant headings
result = pd.merge(trips_df[['DATE', 'LINEID','ROUTEID','DIRECTION', 'ACTUALTIME_SECONDS', 'HOUR', 'UNIQUEID']],
                  leavetimes_df[['PROGRNUMBER','STOPPOINTID','ACTUALTIME_ARR','ACTUALTIME_DEP', 'DWELL_TIME', 'UNIQUEID']],
                  on='UNIQUEID', 
                 how='left')

In [10]:
result.to_csv("Result_check.csv")

In [7]:
result.head()

,DATE,LINEID,ROUTEID,DIRECTION,ACTUALTIME_SECONDS,HOUR,UNIQUEID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,DWELL_TIME
0,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,20,2786,69523,69543,20
1,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,21,2787,69596,69596,0
2,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,56,2797,71063,71074,11
3,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,57,414,71104,71114,10
4,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,58,415,71132,71132,0


In [8]:
stops_df.head()

,pk_id,LINEID,ROUTEID,DIRECTION,TOTAL_STOPS
0,0,1,1_37,1,42
1,1,1,1_38,1,21
2,2,1,1_39,1,23
3,3,1,1_40,2,22
4,4,1,1_41,2,22


In [8]:
result = pd.merge(result,
                  stops_df[['ROUTEID','TOTAL_STOPS']],
                 on='ROUTEID', 
                 how='left')

In [10]:
result.head()

,DATE,LINEID,ROUTEID,DIRECTION,ACTUALTIME_SECONDS,HOUR,UNIQUEID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,DWELL_TIME,TOTAL_STOPS
0,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,20,2786,69523,69543,20,64.0
1,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,21,2787,69596,69596,0,64.0
2,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,56,2797,71063,71074,11,64.0
3,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,57,414,71104,71114,10,64.0
4,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,58,415,71132,71132,0,64.0


In [11]:
distance_df.head()

,Unnamed: 0,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,CURR_STOPID,STOPNAME,STOPLAT,STOPLON,DISTANCE_TRAVELLED,previous_bus_id,previous_stop_dist,PREV_STOPID,DIST_BETWEEN
0,558649,102,102_10,2,2,8240DB003641,"Malahide, Castle Downs Road",53.452741,-6.183434,444.079496,102_10,0.000000,8240DB001073,444.079496
1,84251,102,102_10,2,3,8240DB003642,"Malahide, Church of the Sacred Heart",53.455126,-6.181300,744.224795,102_10,444.079496,8240DB003641,300.145300
2,84254,102,102_10,2,4,8240DB003643,"Malahide, Inbher Ide",53.451266,-6.169253,1767.212437,102_10,744.224795,8240DB003642,1022.987641
3,84250,102,102_10,2,5,8240DB003583,"Malahide, Yellow Walls Road",53.449382,-6.163071,2245.491471,102_10,1767.212437,8240DB003643,478.279034
4,84249,102,102_10,2,6,8240DB003584,"Malahide, Presbyterian Church",53.449855,-6.159873,2464.171602,102_10,2245.491471,8240DB003583,218.680132


In [9]:
result = pd.merge(result,
                  distance_df[['ROUTEID', 'PROGRNUMBER', 'DISTANCE_TRAVELLED', 'CURR_STOPID', 'PREV_STOPID', 'DIST_BETWEEN']],
                 on=['ROUTEID', 'PROGRNUMBER'],
                 how='left')

In [14]:
result = pd.merge(result,
                  weather_df[['temp', 'weather_main', 'HOUR', 'DATE']],
                  on=['HOUR', 'DATE'],
                 how='left')

In [15]:
result

,DATE,LINEID,ROUTEID,DIRECTION,ACTUALTIME_SECONDS,HOUR,UNIQUEID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,DWELL_TIME,TOTAL_STOPS,DISTANCE_TRAVELLED,CURR_STOPID,PREV_STOPID,DIST_BETWEEN,temp,weather_main
0,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,20,2786,69523,69543,20,64.0,5899.216795,8220DB002786,8220DB002785,262.771764,5.08,Clouds
1,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,21,2787,69596,69596,0,64.0,6239.363822,8220DB002787,8220DB002786,340.147027,5.08,Clouds
2,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,56,2797,71063,71074,11,64.0,15626.600410,8220DB002797,8220DB002796,281.154762,5.08,Clouds
3,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,57,414,71104,71114,10,64.0,15823.136180,8220DB000414,8220DB002797,196.535775,5.08,Clouds
4,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,58,415,71132,71132,0,64.0,15993.374250,8220DB000415,8220DB000414,170.238069,5.08,Clouds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398162,2018-01-16,37,37_14,1,3820.0,11:00:00,20180116_6114285,52,4893,43722,43740,18,60.0,17081.000000,8240DB004893,8240DB004573,445.000000,3.03,Clouds
2398163,2018-01-16,37,37_14,1,3820.0,11:00:00,20180116_6114285,60,2959,44266,44266,0,60.0,20849.000000,8240DB002959,8240DB004362,563.000000,3.03,Clouds
2398164,2018-01-16,37,37_14,1,3820.0,11:00:00,20180116_6114285,50,7233,43614,43631,17,60.0,16312.000000,8240DB007233,8240DB004825,328.000000,3.03,Clouds
2398165,2018-01-16,37,37_14,1,3820.0,11:00:00,20180116_6114285,40,1673,43205,43222,17,60.0,13273.000000,8240DB001673,8240DB001672,317.000000,3.03,Clouds


In [16]:
# Sort the results to make it more human readable
result['DATE'] = pd.to_datetime(result['DATE'])
result['WEEKDAY'] = result['DATE'].dt.day_name()

In [17]:
result.head()

,DATE,LINEID,ROUTEID,DIRECTION,ACTUALTIME_SECONDS,HOUR,UNIQUEID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,DWELL_TIME,TOTAL_STOPS,DISTANCE_TRAVELLED,CURR_STOPID,PREV_STOPID,DIST_BETWEEN,temp,weather_main,WEEKDAY
0,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,20,2786,69523,69543,20,64.0,5899.216795,8220DB002786,8220DB002785,262.771764,5.08,Clouds,Monday
1,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,21,2787,69596,69596,0,64.0,6239.363822,8220DB002787,8220DB002786,340.147027,5.08,Clouds,Monday
2,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,56,2797,71063,71074,11,64.0,15626.600410,8220DB002797,8220DB002796,281.154762,5.08,Clouds,Monday
3,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,57,414,71104,71114,10,64.0,15823.136180,8220DB000414,8220DB002797,196.535775,5.08,Clouds,Monday
4,2018-01-15,18,18_3,1,2610.0,19:00:00,20180115_6110593,58,415,71132,71132,0,64.0,15993.374250,8220DB000415,8220DB000414,170.238069,5.08,Clouds,Monday


In [18]:
result = result.reindex(columns=['WEEKDAY', 'LINEID', 'ROUTEID', 'HOUR', 'DIRECTION', 'PROGRNUMBER',
                                 'ACTUALTIME_ARR', 'ACTUALTIME_DEP', 'ACTUALTIME_SECONDS', 'DWELL_TIME', 'TOTAL_STOPS',
                                 'CURR_STOPID', 'PREV_STOPID','DIST_BETWEEN', 'temp', 'weather_main', 'DISTANCE_TRAVELLED', 'UNIQUEID'])

In [19]:
result = result.sort_values(by=['UNIQUEID', 'PROGRNUMBER'])

In [20]:
result.head()

,WEEKDAY,LINEID,ROUTEID,HOUR,DIRECTION,PROGRNUMBER,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_SECONDS,DWELL_TIME,TOTAL_STOPS,CURR_STOPID,PREV_STOPID,DIST_BETWEEN,temp,weather_main,DISTANCE_TRAVELLED,UNIQUEID
2179913,Monday,37,37_14,06:00:00,1,1,24963,24963,3216.0,0,60.0,NaN,NaN,NaN,9.0,Drizzle,NaN,20180115_6105667
2179914,Monday,37,37_14,06:00:00,1,2,24964,24964,3216.0,0,60.0,8220DB000783,8220DB007340,179.0,9.0,Drizzle,179.0,20180115_6105667
2179915,Monday,37,37_14,06:00:00,1,3,24979,24979,3216.0,0,60.0,8220DB000784,8220DB000783,171.0,9.0,Drizzle,350.0,20180115_6105667
2179916,Monday,37,37_14,06:00:00,1,4,25011,25011,3216.0,0,60.0,8220DB000785,8220DB000784,278.0,9.0,Drizzle,628.0,20180115_6105667
2179971,Monday,37,37_14,06:00:00,1,5,25061,25061,3216.0,0,60.0,8220DB000786,8220DB000785,489.0,9.0,Drizzle,1117.0,20180115_6105667


In [21]:
result.isna().sum()

WEEKDAY                    0
LINEID                     0
ROUTEID                    0
HOUR                       0
DIRECTION                  0
PROGRNUMBER                0
ACTUALTIME_ARR             0
ACTUALTIME_DEP             0
ACTUALTIME_SECONDS    264933
DWELL_TIME                 0
TOTAL_STOPS              880
CURR_STOPID           240480
PREV_STOPID           240592
DIST_BETWEEN          240480
temp                       0
weather_main               0
DISTANCE_TRAVELLED    240480
UNIQUEID                   0
dtype: int64

In [22]:
result = result.dropna()

In [23]:
result.to_csv('combined_results.csv')

In [24]:
result.shape

(1923540, 18)

In [25]:
result.isna().sum()

WEEKDAY               0
LINEID                0
ROUTEID               0
HOUR                  0
DIRECTION             0
PROGRNUMBER           0
ACTUALTIME_ARR        0
ACTUALTIME_DEP        0
ACTUALTIME_SECONDS    0
DWELL_TIME            0
TOTAL_STOPS           0
CURR_STOPID           0
PREV_STOPID           0
DIST_BETWEEN          0
temp                  0
weather_main          0
DISTANCE_TRAVELLED    0
UNIQUEID              0
dtype: int64